In [1]:
# test itertools.chain
import itertools
fov_shifts=[(1, 1, 1), (1, 0, 0), (0, 1, 0)]
for off in itertools.chain([(0, 0, 0)], fov_shifts):
    print(' off = {}'.format(off))

 off = (0, 0, 0)
 off = (1, 1, 1)
 off = (1, 0, 0)
 off = (0, 1, 0)


In [2]:
import h5py
import numpy as np

/home/hzh/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# groundtruth
# './data/groundtruth.h5:stack'
gt_file = 'third_party/neuroproof_examples/validation_sample/groundtruth.h5:stack' 
path, dataset=gt_file.split(':')
print(path, dataset)

# dataset
f = h5py.File(path, 'r')
segm = f[dataset]
print(' shape: {}, minv: {}, maxv: {}'.format(segm.shape, np.amin(segm), np.amax(segm)) )
print(' type:{}, value:{}'.format(np.dtype(segm), segm[0, 50, 50]) )

third_party/neuroproof_examples/validation_sample/groundtruth.h5 stack
 shape: (520, 520, 520), minv: 2, maxv: 6210
 type:int64, value:16


In [6]:
# data
raw_file='third_party/neuroproof_examples/validation_sample/grayscale_maps.h5:raw'
path, dataset=raw_file.split(':')
print(path, dataset)
# dataset
f = h5py.File(path, 'r')
raw = f[dataset]
print(' shape: {}, minv: {}, maxv: {}'.format(raw.shape, np.amin(raw), np.amax(raw)))
print(' type:{}, value:{}'.format(np.dtype(raw), raw[0, 50, 50]) )

third_party/neuroproof_examples/validation_sample/grayscale_maps.h5 raw
 shape: (520, 520, 520), minv: 20, maxv: 221
 type:uint8, value:141


In [7]:
import neuroglancer
import webbrowser

viewer = neuroglancer.Viewer()

In [12]:
#
segm = segm[...].astype('uint64')
raw = raw[...]
with viewer.txn() as s:
    s.layers['ground_truth'] = neuroglancer.SegmentationLayer(
        source=neuroglancer.LocalVolume(segm, )
    )
    s.layers['raw'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(raw, )
    )
print(viewer.state)
print(viewer)

ViewerState({"layers": {"ground_truth": {"source": "python://064d16efc200c0603d358f1e173640fe2f7a65e9.e02b0d05c8ce5ff00eacf798012d594f1dec0249", "type": "segmentation"}, "raw": {"source": "python://064d16efc200c0603d358f1e173640fe2f7a65e9.4ba4390a4bdc4d9ca5b82548519b2bcddebe0732", "type": "image"}}, "navigation": {"pose": {"position": {"voxelSize": [1, 1, 1], "voxelCoordinates": [260, 260, 260]}}, "zoomFactor": 1}, "perspectiveOrientation": [-0.17739614844322205, 0.16468545794487, 0.0028907875530421734, 0.9702581763267517], "perspectiveZoom": 8})
http://127.0.0.1:46586/v/064d16efc200c0603d358f1e173640fe2f7a65e9/


In [13]:
# display in the web browser
webbrowser.open_new(viewer.get_viewer_url())

True

In [ ]:
import segyio

### ---- Functions for Input data(SEG-Y) formatting and reading ----
# Make a function that decompresses a segy-cube and creates a numpy array, and
# a dictionary with the specifications, like in-line range and time step length, etc.
def segy_decomp(segy_file, plot_data = False, read_direc='xline', inp_res = np.float64):
    # segy_file: filename of the segy-cube to be imported
    # plot_data: boolean that determines if a random xline should be plotted to test the reading
    # read_direc: which way the SEGY-cube should be read; 'xline', or 'inline'
    # inp_res: input resolution, the formatting of the seismic cube (could be changed to 8-bit data)

    # Make an empty object to hold the output data
    print('Starting SEG-Y decompressor')
    output = segyio.spec()

    # open the segyfile and start decomposing it
    with segyio.open(segy_file, "r" ) as segyfile:
        # Memory map file for faster reading (especially if file is big...)
        segyfile.mmap()

        # Store some initial object attributes
        output.inl_start = segyfile.ilines[0]
        output.inl_end = segyfile.ilines[-1]
        output.inl_step = segyfile.ilines[1] - segyfile.ilines[0]

        output.xl_start = segyfile.xlines[0]
        output.xl_end = segyfile.xlines[-1]
        output.xl_step = segyfile.xlines[1] - segyfile.xlines[0]

        output.t_start = int(segyfile.samples[0])
        output.t_end = int(segyfile.samples[-1])
        output.t_step = int(segyfile.samples[1] - segyfile.samples[0])

        # for qc
        print(" SEGY inline/xline/t geometry:")
        print(" ils={}, ile={}, ili={}, ilen={}\n".format(output.inl_start, output.inl_end, 
                                                          output.inl_step,
                                                          segyfile.xline.len))  # WARNING: WEIRD!!!
        print(" xls={}, xle={}, xli={}, xlen={}\n".format(output.xl_start, output.xl_end, 
                                                          output.xl_step,
                                                          segyfile.iline.len))  # WARNING: WEIRD!!!
        print(" ts={}, te={}, ti={}\n".format(output.t_start, output.t_end, output.t_step))

        # Pre-allocate a numpy array that holds the SEGY-cube
        output.data = np.empty((segyfile.xline.len, segyfile.iline.len, \
                               (output.t_end - output.t_start)//output.t_step+1), 
                               dtype = np.float32)

        # Read the entire cube line by line in the desired direction
        if read_direc == 'inline':
            # Potentially time this to find the "fast" direction
            #start = time.time()
            for il_index in range(segyfile.xline.len):     # WARNING: WEIRD!!!
                output.data[il_index,:,:] = segyfile.iline[segyfile.ilines[il_index]]
            #end = time.time()
            #print(end - start)

        elif read_direc == 'xline':
            # Potentially time this to find the "fast" direction
            #start = time.time()
            for xl_index in range(segyfile.iline.len):    # WARNING: WEIRD!!!
                output.data[:,xl_index,:] = segyfile.xline[segyfile.xlines[xl_index]]
            #end = time.time()
            #print(end - start)

        elif read_direc == 'full':
            ## NOTE: 'full' for some reason invokes float32 data
            # Potentially time this to find the "fast" direction
            #start = time.time()
            output.data = segyio.tools.cube(segy_file)
            #end = time.time()
            #print(end - start)
        else:
            print('Define reading direction(read_direc) using either ''inline'', ''xline'', or ''full''')


        # Convert the numpy array to span between -127 and 127 and convert to the desired format
        factor = 127/np.amax(np.absolute(output.data))
        if inp_res == np.float32:
            output.data = (output.data*factor)
        else:
            output.data = (output.data*factor).astype(dtype = inp_res)

        # If sepcified, plot a given x-line to test the read data
        if plot_data:
            # xline = 100
            xline = np.random.randint(output.data.shape[1])

            # Take a given xline
            data = output.data[:,xline,:]
            
            # Plot the read x-line
            plt.imshow(data.T,interpolation="nearest", cmap="gray")
            plt.title(' xline={}'.format(xline))
            plt.colorbar()
            plt.show()


    # Return the output object
    print('Finished using the SEG-Y decompressor')
    return output

# load the data
import os

# Malenov, parameters for training or predicting
file_dir = '/home/hzh/MachineLearning/segmentation/MalenoV/F3_seismic_data_plus_machine_learning/F3_seismic_data/'
filename='F3_entire.segy'    # name of the segy-cube(s) with data , separate by comma 'volume' for additional volumes
inp_res = np.float32    # formatting of the input seismic (e.g. np.int8 for 8-bit data, np.float32 for 32-bit data, etc)
cube_incr = 32    # number of increments in each direction to create a training cube

# SEGY iline/xline/t dimensions:
ils=100; ile=750; ili=1
xls=300; xle=1250; xli=1
ts=4; te=1848; ti=4

# 
segyfile = os.path.join(file_dir, filename)
segy_obj = segy_decomp(segy_file = segyfile, plot_data = False, read_direc = 'full', inp_res = np.float32)


# display the data
import neuroglancer
import webbrowser

viewer = neuroglancer.Viewer()

#
raw = segy_obj
with viewer.txn() as s:
    s.layers['raw'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(raw, )
    )
print(viewer.state)
print(viewer)

# display in the web browser
webbrowser.open_new(viewer.get_viewer_url())